# Kernel Calculation Test
This notebook is used to test the functionality of the kernel calc methods that (if working) will be rewritten in Python script.

In [1]:
import torch
import resources as rs

Resources Loaded


### Loading Model Data

In [3]:
model_path = 'ex_models.pt'
state_dict = torch.load(model_path, map_location=torch.device('cpu')) # MODEL

### Unpacking the params

**NOTE:** 
The model was trained on the MNIST digits of 0 1

Unpacking the features params

In [4]:
import torch.nn as nn

# get the weights and biases of the quantized model (for the features layer)
f_weights_quant = state_dict['features.hidden_layer._packed_params._packed_params'][0]
f_bias_quant = state_dict['features.hidden_layer._packed_params._packed_params'][1]

# dequantize the weights and biases
f_weights_float = torch.dequantize(f_weights_quant)
f_bias_float = torch.dequantize(f_bias_quant)
# print the float values of weights and biases
print(f_weights_float)
print(f_bias_float)

tensor([[ 0.0212, -0.0311, -0.0050,  ..., -0.0343, -0.0193,  0.0000],
        [ 0.0112, -0.0050,  0.0149,  ..., -0.0093, -0.0106,  0.0336],
        [-0.0324, -0.0006, -0.0062,  ...,  0.0311,  0.0137, -0.0324],
        ...,
        [-0.0305,  0.0168, -0.0293,  ...,  0.0237, -0.0280, -0.0112],
        [-0.0349,  0.0056, -0.0174,  ..., -0.0093, -0.0031,  0.0212],
        [ 0.0349,  0.0168, -0.0206,  ..., -0.0149,  0.0044, -0.0187]])
Parameter containing:
tensor([ 0.0235, -0.0103, -0.0210,  ..., -0.0244,  0.0028, -0.0313],
       grad_fn=<NotImplemented>)


Unpacking the readout params.

In [5]:
# get the weights and biases of the quantized model (for the readout layer)
r_weights_quant = state_dict['readout._packed_params._packed_params'][0]
r_bias_quant = state_dict['readout._packed_params._packed_params'][1]

# dequantize the weights and bises
r_weights_float = torch.dequantize(r_weights_quant)
r_bias_float = torch.dequantize(r_bias_quant)

print(r_weights_float)
print(r_bias_float)

tensor([[ 0.0219,  0.0389,  0.0024,  ..., -0.0073,  0.0182, -0.0158],
        [-0.0195, -0.0219, -0.0024,  ...,  0.0073, -0.0170,  0.0134]])
Parameter containing:
tensor([0.4623, 0.4824], grad_fn=<NotImplemented>)


### Manually updating the model

In [6]:
model = rs.NN()
model

NN(
  (features): Sequential(
    (hidden_layer): Linear(in_features=784, out_features=2048, bias=True)
    (hidden_activation): ReLU()
  )
  (readout): Linear(in_features=2048, out_features=2, bias=True)
)

In [7]:
params = list(model.parameters())
params

[Parameter containing:
 tensor([[ 0.0206, -0.0063, -0.0147,  ..., -0.0132, -0.0048, -0.0265],
         [-0.0117, -0.0267, -0.0291,  ..., -0.0133, -0.0140, -0.0113],
         [-0.0251, -0.0010, -0.0148,  ...,  0.0110,  0.0352,  0.0148],
         ...,
         [-0.0261,  0.0334, -0.0274,  ...,  0.0002, -0.0220,  0.0227],
         [-0.0134, -0.0297, -0.0057,  ...,  0.0053, -0.0297,  0.0346],
         [-0.0099, -0.0183, -0.0262,  ..., -0.0054, -0.0070, -0.0210]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0187, -0.0033,  0.0111,  ...,  0.0338, -0.0140, -0.0327],
        requires_grad=True),
 Parameter containing:
 tensor([[ 0.0040, -0.0104, -0.0056,  ..., -0.0114,  0.0157, -0.0006],
         [ 0.0170, -0.0042, -0.0209,  ...,  0.0210, -0.0009, -0.0077]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0035, -0.0003], requires_grad=True)]

In [8]:
params[0].data = f_weights_float
params[1].data = f_bias_float
params[2].data = r_weights_float
params[3].data = r_bias_float

### Loading MNIST dataset

In [9]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
def mnist_dataset(batch_size, train=True, values=list(range(10))):
    # Initializing MNIST data set.
    dataset = datasets.MNIST(root='dataset/', train=train, transform=transforms.ToTensor(), download=True)

    targets_list = dataset.targets.tolist()
    values_index = [i for i in range(len(dataset)) if targets_list[i] in values]

    # Creating a subset of ### MNIST targets.
    subset = torch.utils.data.Subset(dataset, values_index)
    loader = DataLoader(dataset=subset, shuffle=True)

    return loader

In [10]:
MNIST = rs.mnist_dataset(batch_size=0, train=True, values=[0,1])
data, targets = next(iter(MNIST))

Next, in order to perform the CKA calc. we will need to reshape the data into a batch_size X features tenso (12665, 784).

In [11]:
data = torch.squeeze(data, dim=1)
data = data.view(data.size(0), -1)
data.shape

torch.Size([12665, 784])

### CKA Calc.
At this point we can now calculate the CKA for the model state.

In [12]:
model.features(data).shape

torch.Size([12665, 1024])

In [13]:
targets.shape

torch.Size([12665])

#TODO using this notebook create a python script that will do this for each of the model states (135 * 512)...

### Accuracy

In [14]:
import torch.optim as optim

In [15]:
device = torch.device('cpu')
loss = nn.MSELoss()
model.eval()
losses = rs.train(MNIST, device, model, loss, values=[0, 1], backwards=False, record_loss=True)

In [16]:
losses

8.250704013335053e-06

In [17]:
import time

### CKA Test.
Defining Center Kernel Alignment functions.

In [75]:
def kernel_calc(y, phi):
    y = torch.t(torch.unsqueeze(y, -1))
    
    start = time.time()
    K1 = torch.matmul(torch.t(y), y)
    
    K1c = kernel_centering(K1.float())

    K2 = torch.mm(phi, torch.t(phi))
    
    K2c = kernel_centering(K2)
    end = time.time()

    return kernel_alignment(K1c, K2c)


def frobenius_product(K1, K2):
    return torch.sum(K1 * K2)


def kernel_alignment(K1, K2):
    inner = frobenius_product(K1, K2) 
    mag_norm = ((torch.norm(K1, p='fro') * torch.norm(K2, p='fro')))
    print(mag_norm)
    return inner / mag_norm


def kernel_centering(K):
    row_means = K.mean(dim=1, keepdim=True)
    col_means = K.mean(dim=0, keepdim=True)
    total_mean = K.mean()
    
    return K - row_means - col_means + total_mean

In [76]:
start = time.time()
cka = kernel_calc(targets, model.features(data))
end = time.time()
print(f"CKA : {cka} |TOTAL TIME: {end - start}s")

tensor(1.2270e+08, grad_fn=<MulBackward0>)
CKA : 0.950365424156189 |TOTAL TIME: 7.190279006958008s


### New CKA Function

In [20]:
phi = model.features(data)
y = torch.unsqueeze(targets.T, -1)

/tmp/ipykernel_38110/39957181.py:2: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525493953/work/aten/src/ATen/native/TensorShape.cpp:3277.)
  y = torch.unsqueeze(targets.T, -1)


The new methods potentially allows us to ignore the centering calculation.

In [21]:
# New Method
start = time.time()
v = phi.T.matmul(y.float())
inner = v.T @ v
end = time.time()
print("NEW METHOD")
print(f"INNER = {inner.item()} | TIME = {end - start}s")

NEW METHOD
INNER = 406006848.0 | TIME = 0.00432586669921875s


In [22]:
# Old Method
start = time.time()
K1 = y @ y.T
K1 = K1.float()
K2 = phi @ phi.T
inner = torch.trace(torch.mm(K2, torch.t(K1)))
end = time.time()
print("OLD METHOD")
print(f"INNER = {inner} | TIME = {end - start}s")

OLD METHOD
INNER = 406006848.0 | TIME = 12.153848886489868s


According to this new finding we can decrease computation time for the frobenius product by 3307.0 multiplier!!

### Translating this into HSIC

In [23]:
def HSIC(K1, K2):
    num = torch.trace(torch.mm(K2, K1.T))
    den = (len(K2) - 1) * (len(K1) - 1)
    return num / den

In [24]:
start = time.time()
test = HSIC(K1, K2)
end = time.time()
print("CLASSIC HSIC")
print(f"HSCI = {test} | TIME = {end - start}s")

CLASSIC HSIC
HSCI = 2.5315794944763184 | TIME = 10.108643054962158s


In [25]:
start = time.time()
v = phi.T.matmul(y.float())
test = v.T @ v / (((len(y) - 1) * (len(phi) - 1)))
end = time.time()
print("NEW HSIC")
print(f"NEW HSIC = {test.item()} | TIME = {end - start}s")

NEW HSIC
NEW HSIC = 2.5315794944763184 | TIME = 0.0043621063232421875s


It's of no suprize here that the old methods is much much slower to the same scale as previous.

### Implementing This Findining Into CKA

In [44]:
def cka(y, phi):
    
    # Centering y
    n_y = len(y)
    ones_y = torch.ones(n_y, 1)
    yc = torch.eye(n_y) - (ones_y @ ones_y.T @ y) / n_y
    
    # Centering phi
    n_phi = len(phi)
    ones_phi = torch.ones(n_phi, 1)
    phic = torch.eye(n_phi) - (ones_phi @ ones_phi @ y) / n_phi
    
    # CKA 
    # Numerator HSIC(K_y, K_phi)
    upper = (phic.T @ yc).T @ (phic.T @ yc) / torch.sqrt((n_y - 1)*(n_phi -1))
    # Demonerator sqrt(HSIC(K_y, K_y) * HSIC(K_phi, K_phi))
    lower = torch.sqrt()

In [85]:
# Centering y
y = y.float()
n_y = len(y)
ones_y = torch.ones(n_y, 1)
yc = torch.ones([12665, 1]) - (ones_y @ ones_y.T @ y) / n_y

In [86]:
# Centering phi
n_phi = len(phi)
ones_phi = torch.ones(n_phi, 1)
phic = torch.eye(n_phi) - (ones_phi @ ones_phi.T @ y) / n_phi

### Creating CKA function with this new method

In [97]:
import math

In [103]:
v = phic.T.matmul(yc)
u = (v.T @ v)

In [106]:
d = 1.2270e+08
cka = u / d

In [108]:
cka

tensor([[1025.8938]])